# Assorted helpers

In [ ]:
#| default_exp helpers

In [ ]:
#| export

from web3 import Web3, constants
from typing import List
from decimal import Decimal
from datetime import datetime, timedelta
from dataclasses import dataclass

In [ ]:
#| export

def normalize_address(address: str) -> str: return Web3.to_checksum_address(address.lower())

ADDRESS_ZERO = constants.ADDRESS_ZERO
MAX_UINT256 = Web3.to_int(hexstr='0x' + 'f' * 64)

# def is_address(value: str) -> bool:
#     return Web3.is_address(value)


# def cache_in_seconds(seconds: int):
#     return alru_cache(ttl=seconds)

def chunk(list_to_chunk: List, n: int):
    for i in range(0, len(list_to_chunk), n):
        yield list_to_chunk[i : i + n]


def amount_to_k_string(amount: float) -> str:
    """Turns 2000 to "2K" """
    return f"{round(amount/1000, 2)}K"


def format_currency(value: float, symbol: str = "$", prefix: bool = True) -> str:
    v = "{:0,.2f}".format(value)
    return f"{symbol}{v}" if prefix else f"{v} {symbol}"


def format_percentage(value: float) -> str:
    return "{:0,.2f} %".format(value)


def amount_to_m_string(amount: float) -> str:
    """Turns 2000000 to "2M" """
    return f"{round(amount/1000000, 2)}M"

In [ ]:
#| export

def float_to_uint256(amount: float, decimals: int = 18) -> int:
    """Convert float to uint256 with decimal scaling"""
    # Convert float to Decimal for precision
    amount_decimal = Decimal(str(amount))
    # Scale by decimals
    scaled_amount = amount_decimal * Decimal(10 ** decimals)
    # Convert to integer
    return int(scaled_amount)

In [ ]:
float_to_uint256(1.0, 18)

1000000000000000000

In [ ]:
#| export

def get_future_timestamp(deadline_minutes: float) -> int:
    """Convert minutes from now to future unix timestamp"""
    future_time = datetime.now() + timedelta(minutes=deadline_minutes)
    return int(future_time.timestamp())

In [ ]:
get_future_timestamp(5)

1742218407

In [ ]:
#| export

def apply_slippage(amount: int, slippage: float) -> int:
    if slippage < 0 or slippage > 1: raise ValueError("Slippage must be between 0 and 1")
    return int(amount * (1 - slippage))

## Graph department

This is where all the nerds hang out

In [ ]:
#| export
# Claude 3.7 sonnet made this

from dataclasses import dataclass
import networkx as nx

@dataclass
class Pair:
    token0: str
    token1: str

def find_all_paths(pairs: List[Pair], start_token, end_token, cutoff=3):
    # Create graph
    G = nx.Graph()
    for pair in pairs: G.add_edge(pair.token0, pair.token1)
    
    # Find all simple paths
    return [p for p in nx.all_simple_paths(G, source=start_token, target=end_token, cutoff=cutoff)]


In [ ]:
 # Example pairs - including a cycle
pairs = [
    Pair("A", "B"),
    Pair("B", "C"),
    Pair("A", "D"),
    Pair("D", "C"),
    Pair("B", "D"),
    Pair("C", "E"),
    Pair("E", "A")
]

# Find paths from A to C
paths = find_all_paths(pairs, "A", "C")

print(f"All paths from A to C:")
for i, path in enumerate(paths):
    print(f"Path {i+1}: {' -> '.join(path)}")
    
# Let's also test with a different example involving a cycle
cycle_pairs = [
    Pair("X", "Y"),
    Pair("Y", "Z"),
    Pair("Z", "X")
]

# Find paths from X to Z
cycle_paths = find_all_paths(cycle_pairs, "X", "Z")

print(f"\nAll paths from X to Z in a cyclical graph:")
for i, path in enumerate(cycle_paths):
    print(f"Path {i+1}: {' -> '.join(path)}")

All paths from A to C:
Path 1: A -> B -> C
Path 2: A -> B -> D -> C
Path 3: A -> D -> C
Path 4: A -> D -> B -> C
Path 5: A -> E -> C

All paths from X to Z in a cyclical graph:
Path 1: X -> Y -> Z
Path 2: X -> Z


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()